# 선형 회귀분석

### 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os, joblib
import numpy as np
import pandas as pd

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 압축 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price_Prep.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
dm = pd.get_dummies(data = df['FuelType'], drop_first = True)

In [ ]:
# df의 오른쪽에 더미변수 dm을 추가합니다.
df = pd.concat(objs = [df, dm], axis = 1)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df에서 FuelType을 삭제합니다.
df = df.drop(labels = ['FuelType'], axis = 1)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 목표변수 열이름을 설정합니다.
yvar = 'Price'

In [ ]:
# 입력변수 행렬 X와 목표변수 벡터 y를 생성합니다.
X = df.drop(labels = [yvar], axis = 1)
y = df[yvar]

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [ ]:
# 훈련셋의 목표변수 평균을 확인합니다.
y_tr.mean()

In [ ]:
# 시험셋의 목표변수 평균을 확인합니다.
y_te.mean()

### 다중선형 회귀모형 적합

In [ ]:
# 관련 라이브러리를 호출합니다.
import statsmodels.api as sm

In [ ]:
# 다중선형 회귀모형을 반환하는 함수를 정의합니다.
def ols(y, X):
    model = sm.OLS(endog = y, exog = X)
    return model.fit()

In [ ]:
# 훈련셋 입력변수에 y절편의 역할을 수행할 상수 1을 추가합니다.
X_tr = sm.add_constant(data = X_tr)

In [ ]:
# 훈련셋 입력변수의 처음 5행을 출력합니다.
X_tr.head()

In [ ]:
# 시험셋 입력변수에도 상수 1을 추가합니다.
X_te = sm.add_constant(data = X_te)

In [ ]:
# 훈련셋으로 다중선형 회귀모형을 적합합니다.
fit1 = ols(y = y_tr, X = X_tr)

### 다중선형 회귀모형 적합 결과 확인

In [ ]:
# 다중선형 회귀모형 적합 결과를 확인합니다.
fit1.summary()

### 목표변수의 추정값 생성

In [ ]:
# 훈련셋의 목표변수 추정값을 생성합니다.
y_tr_pred1 = fit1.predict(exog = X_tr)

In [ ]:
# 훈련셋의 실제값과 추정값으로 데이터프레임을 생성하고 값을 비교합니다.
pd.DataFrame(data = {'Real': y_tr, 'Pred': y_tr_pred1})

In [ ]:
# 시험셋의 목표변수 추정값을 생성합니다.
y_te_pred1 = fit1.predict(exog = X_te)

In [ ]:
# 시험셋의 실제값과 추정값으로 데이터프레임을 생성하고 값을 비교합니다.
pd.DataFrame(data = {'Real': y_te, 'Pred': y_te_pred1})

### 회귀모형 성능 평가

In [ ]:
# 관련 모듈을 호출합니다.
import HelloDataScience as hds

In [ ]:
# 훈련셋의 실제값과 추정값으로 선형 회귀모형의 성능지표를 비교합니다.
hds.regmetrics(y_true = y_tr, y_pred = y_tr_pred1)

In [ ]:
# 시험셋의 실제값과 추정값으로 선형 회귀모형의 성능지표를 비교합니다.
hds.regmetrics(y_true = y_te, y_pred = y_te_pred1)

### [참고] MSE, MAE, MAPE 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.metrics import *

In [ ]:
# 시험셋의 MSE를 출력합니다.
mean_squared_error(y_true = y_te, y_pred = y_te_pred1)

In [ ]:
# 시험셋의 MAE를 출력합니다.
mean_absolute_error(y_true = y_te, y_pred = y_te_pred1)

In [ ]:
# 시험셋의 MAPE를 출력합니다.
mean_absolute_percentage_error(y_true = y_te, y_pred = y_te_pred1)

### 회귀계수  유의성 검정 결과로 변수 선택

In [ ]:
# 회귀계수의 유의성 검정을 통과하지 못한 입력변수명으로 리스트를 생성합니다.
cols = ['HP', 'MetColor', 'Doors']

In [ ]:
# 훈련셋과 시험셋에서 회귀계수의 유의성 검정을 통과하지 못한 변수를 제거합니다.
X_tr = X_tr.drop(labels = cols, axis = 1)
X_te = X_te.drop(labels = cols, axis = 1)

In [ ]:
# 변경된 훈련셋으로 다중선형 회귀모형을 적합합니다.
fit2 = ols(y = y_tr, X = X_tr)

In [ ]:
# 다중선형 회귀모형 적합 결과를 확인합니다.
fit2.summary()

### 다중선형 회귀모형의 적합도 확인

In [ ]:
# 다중선형 회귀모형의 AIC를 비교합니다.
print(fit1.aic)
print(fit2.aic)

In [ ]:
# 다중선형 회귀모형의 Adj. R-squared를 비교합니다.
print(fit1.rsquared_adj)
print(fit2.rsquared_adj)

### 분산팽창지수 확인

In [ ]:
# 훈련셋의 입력변수별 분산팽창지수를 출력합니다.
hds.vif(X = X_tr)

In [ ]:
# 다중공선성 입력변수가 있다고 가정하고 훈련셋에서 삭제합니다.
X_tr1 = X_tr.drop(labels = ['Petrol'], axis = 1)

In [ ]:
# 훈련셋의 입력변수별 분산팽창지수를 다시 출력합니다.
hds.vif(X = X_tr1)

In [ ]:
# 변경된 훈련셋으로 다중선형 회귀모형을 적합합니다.
fit3 = ols(y = y_tr, X = X_tr1)

In [ ]:
# 다중선형 회귀모형의 적합 결과를 확인합니다.
fit3.summary()

### 표준화 회귀계수 확인

In [ ]:
# 다중선형 회귀모형의 회귀계수를 출력합니다.
fit2.params

In [ ]:
# 입력변수의 표준편차를 목표변수의 표준편차로 나눈 값을 출력합니다.
X_tr.std() / y_tr.std()

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = fit2.params * (X_tr.std() / y_tr.std())

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순으로 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 회귀모형 성능 평가

In [ ]:
# 시험셋의 목표변수 추정값을 생성합니다.
y_te_pred2 = fit2.predict(exog = X_te)

In [ ]:
# 시험셋의 실제값과 추정값으로 데이터프레임을 생성하고 값을 비교합니다.
pd.DataFrame(data = {'Real': y_te, 'Pred': y_te_pred2})

In [ ]:
# 시험셋의 실제값과 추정값으로 fit1의 성능지표를 비교합니다.
hds.regmetrics(y_true = y_te, y_pred = y_te_pred1)

In [ ]:
# 시험셋의 실제값과 추정값으로 fit2의 성능지표를 비교합니다.
hds.regmetrics(y_true = y_te, y_pred = y_te_pred2)

## End of Document